# 03b - Vertex AI + BQML - Online Predictions with BQML Models

Models built with BigQuery ML (BQML), like the one in (03a), can also be exported for use and deployment outside of BigQuery.  A Vertex AI Endpoint can be used for online predictions with an exported model.  This demonstration shows the process of exporting and deploying a BQML model with Vertex AI.

**Prerequisites:**

-  03a - BigQuery Machine Learning (BQML) - Machine Learning with SQL

**Overview:**

-  Export the BigQuery ML model built in (03a) to a GCS bucket URI
   -  EXPORT MODEL …
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict

**Resources:**

-  [Export formats for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/exporting-models)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)

**Related Training:**

-  todo

---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/slide_15.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/slide_16.png">

---
## Setup

inputs:

In [2]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'fraud'
NOTEBOOK = '03b'

# Resources
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [3]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
params = {"URI": URI}
DIR = f"temp/{NOTEBOOK}"

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [7]:
export = bigquery.query(query = f"EXPORT MODEL {DATANAME}.{DATANAME}_lr OPTIONS(URI = '{URI}')")

In [8]:
export.result()

---
## Serving

### Upload The Model

In [9]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/691911073727/locations/us-central1/models/420444450368520192/operations/8958485117194469376
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/691911073727/locations/us-central1/models/420444450368520192
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/691911073727/locations/us-central1/models/420444450368520192')


In [10]:
model.display_name

'03b_fraud_20210922110523'

### Create An Endpoint

In [11]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/691911073727/locations/us-central1/endpoints/6134729525222703104/operations/3473100771057205248
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/691911073727/locations/us-central1/endpoints/6134729525222703104
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/691911073727/locations/us-central1/endpoints/6134729525222703104')


In [12]:
endpoint.display_name

'03b_fraud_20210922110523'

### Deploy Model To Endpoint

In [13]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/691911073727/locations/us-central1/models/420444450368520192 to Endpoint : projects/691911073727/locations/us-central1/endpoints/6134729525222703104
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/6134729525222703104/operations/590797009540087808
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/691911073727/locations/us-central1/endpoints/6134729525222703104


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [25]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [26]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,75176,1.235603,0.041383,0.675286,0.836279,-0.675016,-0.657342,-0.154209,-0.067491,0.602617,...,0.088164,0.396205,0.324557,0.182930,-0.017115,0.014979,0.0,0,6f40e111-2131-4031-aef1-268b2daf02a3,TEST
1,112225,-0.285756,0.965688,2.147689,2.838137,1.104026,1.462921,-0.835272,-0.409875,-0.810586,...,-1.222639,0.044689,1.324343,-0.077670,0.128146,0.179651,0.0,0,c9bdeaf0-79d4-4faf-8aaa-14127a0e3513,TEST
2,113420,1.890283,0.241366,-0.165823,4.068924,-0.146807,0.140339,-0.258090,0.084852,-0.056567,...,0.097449,0.019180,0.060621,0.136349,-0.010745,-0.049814,0.0,0,611f4879-490d-4ab5-9e4f-c7944d6165eb,TEST
3,121910,-1.227268,1.555572,1.245848,4.071686,1.154573,2.058276,-1.179951,-2.211140,-2.248168,...,-0.380109,-1.485002,0.237729,0.524141,-0.037543,0.121962,0.0,0,3212444f-1412-49ae-adfa-a1a0ca54d8c5,TEST


In [30]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET,'splits'])]].to_dict(orient='records')[0]
#newob

In [31]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [32]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[{'Class_values': ['1', '0'], 'Class_probs': [0.05572191194179966, 0.9442780880582003], 'predicted_Class': ['0']}], deployed_model_id='5471389762138931200', explanations=None)

In [33]:
prediction.predictions[0]#['classes'][np.argmax(prediction.predictions[0]['scores'])]

{'Class_values': ['1', '0'],
 'Class_probs': [0.05572191194179966, 0.9442780880582003],
 'predicted_Class': ['0']}

In [34]:
prediction.predictions[0][f'{VAR_TARGET}_values'][np.argmax(prediction.predictions[0][f'{VAR_TARGET}_probs'])]

'0'

### Get Predictions: REST

In [35]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [36]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    {
      "predicted_Class": [
        "0"
      ],
      "Class_probs": [
        0.055721911941799658,
        0.94427808805820035
      ],
      "Class_values": [
        "1",
        "0"
      ]
    }
  ],
  "deployedModelId": "5471389762138931200"
}


### Get Predictions: gcloud (CLI)

In [37]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'Class_probs': [0.05572191194179966, 0.9442780880582003], 'Class_values': ['1', '0'], 'predicted_Class': ['0']}]


---
## Remove Resources
see notebook "XX - Cleanup"